# Banking Agentic Demo (Tools + Policy RAG)

This notebook is a trimmed, **agent-only** version of *From RAG to AI Agent*.

It implements a simple banking assistant that:
1. Asks for a **customer ID**.
2. Uses a **SQLite** tool to retrieve the customer's products.
3. Uses a **Policy RAG** tool to look up what is allowed/forbidden.
4. Answers questions **only within policy**.

> **Note**: Data and policy are fictitious for demo purposes.

In [ ]:
%pip install -q openai qdrant-client docling fastembed jupyter-chat-widget

## 1) Setup: OpenAI client + chat UI

In [ ]:
import os
import openai
from google.colab import userdata
from jupyter_chat_widget import ChatUI


def get_oai_client():
    """Creates the OpenAI client and initializes chat history."""
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI()
    messages = []
    return client, messages


def display_response(ui, stream):
    """Stream tokens to the UI and return the final text."""
    answer = ""
    ui.rewrite("")
    for token in stream:
        if token.choices[0].delta and token.choices[0].delta.content:
            answer += token.choices[0].delta.content
            ui.rewrite(answer)
    return answer


## 2) Create a tiny SQLite 'core banking' DB (demo)

In [ ]:
import sqlite3
from pathlib import Path

DB_PATH = Path('bank_demo.sqlite')

# Recreate for repeatable demos
if DB_PATH.exists():
    DB_PATH.unlink()

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.execute('''
CREATE TABLE customers (
  customer_id TEXT PRIMARY KEY,
  full_name TEXT,
  segment TEXT
)
''')

cur.execute('''
CREATE TABLE products (
  product_id TEXT PRIMARY KEY,
  customer_id TEXT,
  product_type TEXT,
  nickname TEXT,
  currency TEXT,
  balance REAL,
  credit_limit REAL,
  apr REAL,
  FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
)
''')

cur.executemany(
    "INSERT INTO customers VALUES (?,?,?)",
    [
        ("CUST-1001", "Alicia Mora", "Retail"),
        ("CUST-2002", "Diego Hernández", "SMB"),
    ],
)

cur.executemany(
    "INSERT INTO products VALUES (?,?,?,?,?,?,?,?,?)",
    [
        ("P-CHK-01", "CUST-1001", "checking", "Daily Checking", "USD", 1250.75, None, None),
        ("P-CC-01",  "CUST-1001", "credit_card", "Travel Card", "USD", -230.10, 5000.00, 0.279),
        ("P-SAV-01", "CUST-2002", "savings", "Business Savings", "USD", 18250.00, None, None),
        ("P-LOC-01", "CUST-2002", "line_of_credit", "Working Capital", "USD", -1200.00, 25000.00, 0.185),
    ],
)

conn.commit()
conn.close()

print(f"SQLite demo DB created at: {DB_PATH.resolve()}")


## 3) Policy document (RAG source)
We'll store a short fictitious policy in a markdown file and index it in Qdrant (in-memory).

In [ ]:
from pathlib import Path

POLICY_PATH = Path('bank_policy.md')

POLICY_TEXT = """
# Demo Banking Policy (Fictitious)

## Identity & privacy
- The assistant must **ask for a customer ID** before accessing account information.
- The assistant may **summarize products** (type, nickname, currency, balance, limit, APR) once retrieved.
- The assistant must **not disclose** full account numbers, SSNs, addresses, phone numbers, or any sensitive PII.

## Allowed actions
- Provide explanations about product features: checking, savings, credit cards, lines of credit.
- Explain balances, credit limits, and APR, and what they generally mean.
- Provide general guidance on next steps (e.g., "contact the bank", "visit branch") when the policy forbids direct changes.

## Disallowed actions
- The assistant **cannot** execute transactions (transfers, bill pay, card payments), change credit limits, close accounts, or apply fees.
- The assistant **cannot** provide personalized legal/tax advice.
- The assistant **cannot** reveal internal risk models, fraud thresholds, or security procedures.

## How to respond
- If the user asks for a forbidden action, the assistant must:
  1) Say it cannot perform that action, citing policy rationale.
  2) Offer safe alternatives (e.g., how to contact support).

## Product-specific notes
- Credit card: You may explain minimum payments conceptually but **do not** compute exact fees/interest beyond simple illustrative examples.
- Line of credit: You may describe how utilization works and how APR affects interest.

"""

POLICY_PATH.write_text(POLICY_TEXT.strip(), encoding='utf-8')
print(f"Policy saved to: {POLICY_PATH.resolve()}")


## 4) RAG ingestion (Docling → chunks → Qdrant in-memory)

In [ ]:
from time import sleep
from uuid import uuid4
from qdrant_client import QdrantClient, models
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker


def ingest_documents(ui, paths):
    # Prepare vectordb + embedder
    vdb = QdrantClient(location=":memory:")
    dense_model = "sentence-transformers/all-MiniLM-L6-v2"
    vdb.set_model(dense_model)

    collection_name = "documents"
    vdb.create_collection(
        collection_name=collection_name,
        vectors_config=vdb.get_fastembed_vector_params(),
    )

    points = []
    chunker = HybridChunker()

    for path in paths:
        ui.rewrite(f"[Parsing {path}...]")
        doc = DocumentConverter().convert(source=str(path)).document
        for chunk in chunker.chunk(dl_doc=doc):
            enriched_text = chunker.contextualize(chunk=chunk)
            meta = chunk.meta.export_json_dict()
            points.append(
                models.PointStruct(
                    id=uuid4().hex,
                    payload=meta | {"document": enriched_text},
                    vector={
                        vdb.get_vector_field_name(): models.Document(
                            text=enriched_text, model=dense_model
                        ),
                    },
                )
            )

    vdb.upload_points(collection_name=collection_name, points=points, batch_size=64, wait=True)
    ui.rewrite("[Policy indexed. Ready!]")
    return vdb


def search_policy(ui, vdb, query, limit=8):
    ui.rewrite("[Searching policy...]")
    samples = vdb.query(
        collection_name="documents",
        query_text=query,
        limit=limit,
    )
    ui.rewrite(f"[Found {len(samples)} snippets]")
    sleep(0.5)
    return {
        "role": "user",
        "content": "Relevant policy snippets:\n\n" + "\n\n".join(s.document for s in samples),
    }


## 5) Tool 1: retrieve customer products from SQLite

In [ ]:
import sqlite3


def get_customer_products(customer_id: str):
    """Return customer + products as a JSON-like dict."""
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.execute("SELECT customer_id, full_name, segment FROM customers WHERE customer_id = ?", (customer_id,))
    cust = cur.fetchone()
    if not cust:
        conn.close()
        return {"found": False, "customer_id": customer_id, "error": "Customer not found"}

    cur.execute(
        """
        SELECT product_id, product_type, nickname, currency, balance, credit_limit, apr
        FROM products
        WHERE customer_id = ?
        ORDER BY product_type
        """,
        (customer_id,),
    )
    rows = cur.fetchall()
    conn.close()

    products = []
    for r in rows:
        products.append(
            {
                "product_id": r[0],
                "product_type": r[1],
                "nickname": r[2],
                "currency": r[3],
                "balance": r[4],
                "credit_limit": r[5],
                "apr": r[6],
            }
        )

    return {
        "found": True,
        "customer": {"customer_id": cust[0], "full_name": cust[1], "segment": cust[2]},
        "products": products,
    }


## 6) Agent loop: OpenAI tool-calling + two tools (products + policy search)

The agent must:
- Ask for customer ID if missing.
- Call `get_customer_products` once it has the ID.
- Call `search_policy` to ground decisions (allowed vs forbidden).
- Answer the user and stay within policy.


In [ ]:
import json
import re

# --- Tool schema (for OpenAI tool-calling) ---

get_products_tool = {
    "type": "function",
    "function": {
        "name": "get_customer_products",
        "description": "Retrieve the customer's banking products from the SQLite core banking DB using customer_id.",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "Customer identifier, e.g., CUST-1001",
                }
            },
            "required": ["customer_id"],
        },
    },
}

policy_search_tool = {
    "type": "function",
    "function": {
        "name": "search_policy",
        "description": "Search the policy document for relevant rules and constraints to answer safely.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "What to search in the policy"},
                "limit": {"type": "integer", "description": "How many snippets", "default": 8},
            },
            "required": ["query"],
        },
    },
}


# --- Tool implementations mapping ---

def _tool_get_customer_products(ui, vdb, customer_id: str):
    data = get_customer_products(customer_id)
    # Put tool result into the chat as a tool message (so the LLM can use it)
    return {
        "role": "tool",
        "name": "get_customer_products",
        "content": json.dumps(data, ensure_ascii=False),
    }


def _tool_search_policy(ui, vdb, query: str, limit: int = 8):
    # Reuse the RAG helper but return as a tool message
    msg = search_policy(ui, vdb, query=query, limit=limit)
    return {
        "role": "tool",
        "name": "search_policy",
        "content": msg["content"],
    }


tool_implementations = {
    "get_customer_products": _tool_get_customer_products,
    "search_policy": _tool_search_policy,
}


def query_llm_with_tools(ui, client, vdb, messages, tools):
    ui.rewrite("[Thinking...]")
    stream = client.chat.completions.create(
        model="gpt-5.2",
        messages=messages,
        stream=True,
        tools=tools,
    )

    first_token = next(stream)
    if first_token.choices[0].delta.tool_calls:
        return _invoke_tool(ui, client, vdb, messages, tools, first_token, stream)
    return stream


def _invoke_tool(ui, client, vdb, messages, tools, first_token, stream):
    tool_call = first_token.choices[0].delta.tool_calls[0]
    tool_name = tool_call.function.name
    ui.rewrite(f"[Calling tool: {tool_name}]")

    # Collect JSON args across tokens
    args_json = ""
    for token in stream:
        if token.choices[0].delta.tool_calls:
            args_json += token.choices[0].delta.tool_calls[0].function.arguments

    tool_args = json.loads(args_json or "{}")

    # Execute
    impl = tool_implementations[tool_name]
    tool_msg = impl(ui, vdb, **tool_args)
    messages.append(tool_msg)

    # Continue the loop (LLM may call tools again)
    return query_llm_with_tools(ui, client, vdb, messages, tools)


# --- Helper: try to extract a customer id from user text ---
CUST_ID_RE = re.compile(r"\bCUST-\d{4}\b", re.IGNORECASE)

def extract_customer_id(text: str):
    m = CUST_ID_RE.search(text or "")
    return m.group(0).upper() if m else None


## 7) Run the agent
Try:
- `Hi` → it should ask for the customer ID.
- `My ID is CUST-1001` → it should retrieve products.
- `Can you increase my credit limit?` → it should refuse (policy).
- `Explain my Travel Card APR and what it means` → should answer.


In [ ]:
def run_banking_agent():
    ui = ChatUI()
    client, messages = get_oai_client()

    # Index only the policy doc (you can add more docs later)
    vdb = ingest_documents(ui, [POLICY_PATH])

    system_prompt = """
You are a banking assistant for a demo bank.

Rules:
- Before accessing account/product info, you MUST ask for the customer's ID (format CUST-####).
- When you have an ID, you MUST call the tool get_customer_products to retrieve products.
- For every user question about what you can/can't do, or whether an action is allowed, you MUST call search_policy.
- Follow the policy strictly.
- Never reveal sensitive PII. If asked for forbidden actions, refuse and offer safe alternatives.

Workflow:
1) If you don't have customer_id in conversation state, ask for it.
2) If you have customer_id but haven't loaded products yet, call get_customer_products.
3) Answer the user's question using the retrieved products + policy.
""".strip()

    messages.append({"role": "system", "content": system_prompt})

    # Simple in-memory state
    state = {
        "customer_id": None,
        "products_loaded": False,
    }

    def _handle(query: str):
        # Update ID if user provided one
        cid = extract_customer_id(query)
        if cid:
            state["customer_id"] = cid

        messages.append({"role": "user", "content": query})

        # Enforce the workflow by nudging the LLM with a short developer-style hint
        # (still within standard messages, keeps the notebook structure simple)
        if not state["customer_id"]:
            messages.append({
                "role": "system",
                "content": "The user has not provided a customer ID yet. Ask for it before doing anything else.",
            })
        elif not state["products_loaded"]:
            messages.append({
                "role": "system",
                "content": f"Customer ID is {state['customer_id']}. Call get_customer_products now.",
            })

        stream = query_llm_with_tools(
            ui,
            client,
            vdb,
            messages,
            tools=[get_products_tool, policy_search_tool],
        )

        response = display_response(ui, stream)
        messages.append({"role": "assistant", "content": response})

        # Detect if products were loaded by checking last tool call
        # (simple heuristic: if any tool message with name get_customer_products exists)
        if any(m.get("role") == "tool" and m.get("name") == "get_customer_products" for m in messages[-10:]):
            state["products_loaded"] = True

    ui.connect(lambda q: _handle(q))


run_banking_agent()
